# MEFE - Ejercicio 9, guía 2

### Enunciado:

#### **9)**
Una fabrica produce integrados, de los cuales el 20% son defectuosos y los comercializa en cajas de 10. Un comprador
quiere rechazar las cajas que contengan más de 2 chips defectuosos, es decir, más que la especificación del fabricante.
Para ganar tiempo, en vez de probar todos los chips decide implementar el siguiente test. De cada caja toma 6 chips
al azar: (i) si ninguno es malo, acepta la caja; (ii) si uno solo es malo, revisa el resto de la caja; (iii) si 2 o más son malos, devuelve la caja al fabricante.
- **a)** ¿Qué fracción de las cajas tendrán más de 2 chips defectuosos? [Rta: 0.322]
- **b)** ¿En qué fracción de las cajas deberá probar los 10 integrados? [Rta: 0.3932]
- **c)** ¿Cuál es la probabilidad de que haya 3 chips malos en una caja aceptada? [Rta: 0.0114]

### Solución:

**a)**
Para armar las cajas, fabricante toma uno de sus integrados y lo mete adentro. Luego otro, y lo vuelve a meter en la caja. Si cada caja tiene $N$ integrados, repite esta acción $N$ veces. Con un 20% de chances, cada integrado que agarre el fabricante al armar la caja será defectuoso ($\overline{B}$), y por lo tanto con un 80% de chances, será bueno ($B$).

Entonces la probabilidad de tomar, por ejemplo, 10 integrados malos, es $p^{10}=0.2^{10}$, o 10 buenos sería $(1-p)^{10} = 0.8^{10}$. Pero en general, la probabilidad de sacar $n$ malos y $N-n$ buenos es $p^n (1-p)^{N-n}$. Da igual el orden en el que el fabricante ponga los integrados en la caja, al fin y al cabo la cantidad de integrados malos es la misma si agarró los malos primeros y los buenos después, o al revés. Por eso, hay que multiplicar por la cantidad de formas que hay de sacar $n$ malos y $N-n$ buenos, que es el combinatorio ${N \choose n}$. 

En conclusión, la probabilidad de que una caja con $N$ integrados tenga $n$ malos dado que la probabilidad de poner uno malo en la caja es $p$ es
$$B(n \mid N, p) = {N \choose n} p^n (1-p)^{N-n}$$,
entonces la probabilidad de que tenga 2 o más es dado $N=10$ y $p=0.2$ es
$$\sum_{n=3}^{n=10} {10 \choose n} 0.2^n 0.8^{10-n} = 0.322$$. 

Se puede chequear con código de dos formas:
1. armando las cajas y contando cuántas tienen más de 2 integrados malos
1. haciendo lo mismo pero tomando los valores random directamente de una binomial

In [1]:
import random
import numpy as np
import math

def bin(N, n, p):
    return math.comb(N, n) * p**n * (1-p)**(N-n)

# Parámetros
p = 0.2
N = 10
tolerancia_minima = 2
n_cajas_compradas = 100000

# Experimental
cajas_compradas = np.array([[(random.random() < p)*1 for i in range(N)] for j in range(n_cajas_compradas)])
malos_por_caja = np.count_nonzero(cajas_compradas, axis=1)
cajas_con_mas = np.count_nonzero(malos_por_caja > tolerancia_minima)
print(f"Propoción de cajas armadas con más de {tolerancia_minima} integrados malos:\n{cajas_con_mas/n_cajas_compradas}\n")

# Binomial
malos_por_caja_binomial = np.random.binomial(N, p, size=n_cajas_compradas)
cajas_con_mas_binomial = np.count_nonzero(malos_por_caja_binomial > tolerancia_minima)
print(f"Propoción de cajas binomial con más de {tolerancia_minima} integrados malos:\n{cajas_con_mas_binomial/n_cajas_compradas}\n")

# Teórico
teorico = np.sum([bin(N, n, p) for n in range(3, 11)])
pm3 = teorico
print(f"El resultado teórico es:\n{teorico}")

Propoción de cajas armadas con más de 2 integrados malos:
0.32176

Propoción de cajas binomial con más de 2 integrados malos:
0.32246

El resultado teórico es:
0.32220047360000015


**b)**
El comprador debe probar todos los integrados sólo en el caso en el que sea positiva la condición (ii) de su test, es decir, cuando de los 6 que sacó al azar de la caja, sólo 1 es malo. Sabemos que para que eso pase una caja puede tener a lo sumo 5 integrados malos, porque si tuviera 6 o más seguro saca 2 o más malos al sacar 6 al azar, por lo que descartaría la caja de inmediato. En el otro extremo, la caja debe tener al menos 1 malo, pues si no la aceptaría de inmediato. En general, queremos saber cuál es la probabilidad de sacar $k$ integrados malos al extraer $n$ integrados de una caja que contiene $N$ en total, dado que $m$ de ellos son malos. 

Ningún integrado tiene más probabilidad de ser extraído que otro, así que podemos calcular la probabilidad contando casos favorables sobre casos totales. Acá, la cantidad de casos totales es la cantidad de formas de tomar $n$ objetos de un grupo de $N$, que es ${N \choose n}$. Los casos "favorables" serían los casos en los que $k$ de los $n$ que agarramos son malos, y por lo tanto $n-k$ son buenos. La cantidad de formas de extraer $k$ de los $m$ malos que tiene la caja es ${m \choose k}$, y la cantidad de formas de extraer los $n-k$ buenos restantes es ${N - m \choose n - k}$, por lo que la probabilidad de obtener $k$ malos al extraer $n$ integrados de una caja de $N$ con $m$ malos está dada por la hipergeométrica

$$H(k \mid n, m, N) = \frac{{m \choose k}{N-m \choose n-k}}{{N \choose n}}.$$

Hay que recordar que esta probabilidad tiene en cuenta que hay $m$ malos en la caja, pero eso no es dato para nosotros. Lo que queremos calcular es $P(k=1)$, pero la cuenta de arriba nos dice $P(k \mid m)$. Para obtener lo que queremos hace falta multiplicar por la probabilidad de obtener $m$ malos en una caja, es decir $P(k)=P(k \mid 
m)P(m)$, que por lo que dedujimos en el **a)** es

$$P(k) = {N \choose m} p^m (1-p)^{N-m} \frac{{m \choose k}{N-m \choose n-k}}{{N \choose n}}.$$

En particular, a nosotros nos interesa saber la probabilidad de sacar $k=1$ malo al sacar un subconjunto de $n=6$ de una caja de $N=10$ integrados, dado que puede haber $m=1, 2, 3, 4$ o $5$ integrados malos dentro de ella. Es decir

$$\sum_{m=1}^5 {N \choose m} p^m (1-p)^{N-m} \frac{{m \choose 1}{10-m \choose 6-1}}{{10 \choose 6}} = 0.3932.$$

Ahora para verificar codeamos.

Primero hacemos la cuenta analítica:

In [2]:
# Defino la hipergeométrica:
def hg(k, n, m, N):
    return (math.comb(m, k) * math.comb(N-m, n-k))/math.comb(N, n)

# Constantes
k = 1
n = 6

# Teórico
teorico = np.sum([hg(k, n, m, N)*bin(N, m, p) for m in range(1, 6)])
print(f"El resultado teórico es:\n{teorico}\n")

El resultado teórico es:
0.39321600000000023



Ahora, como las cajas compradas del punto **a)** ya están hechas de forma aleatoria, puedo reutilizarlas:

In [3]:
subconjuntos_test = cajas_compradas[:, :6] # Tomo el subconjunto de integrados en cada caso
malos_por_subconjunto = np.count_nonzero(subconjuntos_test, axis=1) # Cuento la cantidad de malos por subconjunto
subconjuntos_con_k = np.count_nonzero(malos_por_subconjunto == k) 
print(f"Proporcion de cajas en las que hay que testear todos:\n{subconjuntos_con_k/n_cajas_compradas}")

Proporcion de cajas en las que hay que testear todos:
0.39243


La caja tiene que ser aceptada **y** tener 3 integrados (o más). Eso sólo pasa si el test resulta en el caso (i), porque en el caso (iii) es rechazada y en el caso (ii) si es aceptada es porque tiene 1 o 2 malos. Entonces contemos la cantidad de cajas que tienen 3 o más integrados malos, y que a la vez tienen 0 malos en los 6 que salieron para testear. Podemos usar la probabilidad que dedujimos en el **b)** con $k=0$ y el resto de parámetros igual, sumando para $m=3$ y $m=4$, es decir

$$\sum_{m=3}^4 {N \choose m} p^m (1-p)^{N-m} \frac{{m \choose 1}{10-m \choose 6-1}}{{10 \choose 6}} = $$


In [4]:
# Ya que la uso tanto defino la función probabilidad
def prob(m_min, m_max, k, n, N, p):
    return np.sum([hg(k, n, m, N)*bin(N, m, p) for m in range(m_min, m_max+1)])
# Constantes
k = 0
m_min = 3
m_max = 4

# Teórico
teorico = np.sum([hg(k, n, m, N)*bin(N, m, p) for m in range(m_min, m_max)])
print(f"El resultado teórico es:\n{teorico}\n")

El resultado teórico es:
0.006710886400000004



0.5771362304000004
0.6777995263999999
0.32220047360000015
0.007130316800000004
0.005838647779470801
